In [1]:
%config Completer.use_jedi = False

In [2]:
import itertools
import numpy as np

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.autograd import Variable

In [4]:
from datetime import datetime
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [5]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device : %s'%(DEVICE))

device : cuda:0


In [6]:
torch.__version__

'1.9.0+cu111'

In [7]:
from models.AAE import encoder, decoder
from modules.MyDataset import MaskDataset

device : cuda:0


In [8]:
enc = encoder()

In [9]:
z, mu, logvar = enc(torch.zeros((1,3,320,256)).to(DEVICE))
z.shape

torch.Size([1, 80])

In [10]:
dec = decoder()

In [11]:
re_im = dec(z)
re_im.shape

torch.Size([1, 3, 320, 256])

In [12]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator,self).__init__()
        
        self.linear_1 = nn.Linear(80, 40, device = DEVICE)
        self.L_relu_1 = nn.ReLU()
        
        self.linear_2 = nn.Linear(40, 20, device = DEVICE)
        self.L_relu_2 = nn.ReLU()
        
        self.linear_3 = nn.Linear(20, 10, device = DEVICE)
        self.L_relu_3 = nn.ReLU()
        
        self.linear_4 = nn.Linear(10, 1, device = DEVICE)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.linear_1(x)
        x = self.L_relu_1(x)
        
        x = self.linear_2(x)
        x = self.L_relu_2(x)
        
        x = self.linear_3(x)
        x = self.L_relu_3(x)
        
        x = self.linear_4(x)
        x = self.sigmoid(x)
        
        
        return x

In [13]:
D = discriminator()

In [14]:
D.forward(torch.FloatTensor(np.random.randn(1,40)).to(DEVICE)).shape

torch.Size([1, 1])

In [15]:
BATCH_SIZE    = 128
LEARNING_RATE = 0.001
TOTAL_EPOCH   = 100
DATE_TIME     = datetime.now().strftime("%H:%M:%S")

In [16]:
composed = transforms.Compose([
    transforms.ToTensor(),
    lambda img : transforms.functional.crop(img, 80, 50, 320, 256)
])

only_normal_dataset = MaskDataset(
    mode        = 'only_normal',
    train       = True,
    csv_path    = './train/train.csv',
    images_path = './train/images/',
    valid_ratio = 0.1,
    transforms  = composed,
)

In [17]:
data = only_normal_dataset[0]
data[0].shape

torch.Size([3, 320, 256])

In [18]:
dataloader = DataLoader(
    dataset     = only_normal_dataset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
    num_workers = 4,
    drop_last   = True
)
sample_images, sample_labels = next(iter(dataloader))
print(sample_images.shape)
print(sample_labels.shape)

torch.Size([128, 3, 320, 256])
torch.Size([128])


In [19]:
opt_G = optim.Adam(
    itertools.chain(enc.parameters(),dec.parameters()),
    lr = LEARNING_RATE
)
opt_D = optim.Adam(
    itertools.chain(enc.parameters(),dec.parameters()),
    lr = LEARNING_RATE
)


adversarial_loss = torch.nn.MSELoss()
pixcelwise_loss = torch.nn.L1Loss()

In [22]:

writer = SummaryWriter(
    'runs/experiment_%s'%(DATE_TIME)
)

g_step = 0

# Adversarial ground truths
Tensor = torch.cuda.FloatTensor
valid = Variable(Tensor(BATCH_SIZE, 1).fill_(1.0), requires_grad=False)
fake = Variable(Tensor(BATCH_SIZE, 1).fill_(0.0), requires_grad=False)

for ep in range(TOTAL_EPOCH):
    for images, labels in iter(dataloader):
        g_step += 1
        
        enc.train()
        dec.train()
        # Generator
        
        X = images.to(DEVICE)
        latent_vector, mu, logvar = enc(X)
        re_im = dec(latent_vector)
        
        g_loss_val = \
        0.001 * adversarial_loss(D(latent_vector), valid)+\
        0.999 * pixcelwise_loss(re_im, X)
        
        opt_G.zero_grad()
        g_loss_val.backward()
        opt_G.step()
        
        # Discriminator
        
        # Sample noise as discriminator ground truth
        zeros = torch.zeros(latent_vector.shape)
        ones = torch.ones(latent_vector.shape)
        z = torch.normal(zeros, ones)
        z = Variable(labels.unsqueeze(1)*5 + z)
        
        real_loss = adversarial_loss(D(z.to(DEVICE)), valid)
        fake_loss = adversarial_loss(D(latent_vector.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        
        opt_D.zero_grad()
        d_loss.backward()
        opt_D.step()
        
        writer.add_scalar(
            "G_loss",
            g_loss_val,
            g_step
        )
        writer.add_scalar(
            "D_loss",
            d_loss,
            g_step
        )
    
    
    enc.eval()
    dec.eval()
    
    img_grid = torchvision.utils.make_grid(sample_images)
    writer.add_image(
        'original',
        img_grid,
        global_step = g_step
    )
    
    z, mu, logvar = enc(sample_images.to(DEVICE))
    sample_re_im = dec(z)
    img_grid = torchvision.utils.make_grid(sample_re_im)
    writer.add_image(
        'reconstruction',
        img_grid,
        global_step = g_step
    )

    
    
    writer.add_embedding(
        mu,
        metadata = sample_labels,#["female", "male"],
        #label_img=sample_images[:,:,:256,:],
        global_step = g_step
    )
        
    print(ep)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
SAVE_PATH = "./saved_models/encoder_%s.pt"%DATE_TIME

torch.save(enc.state_dict(), SAVE_PATH)

In [ ]:
data_dict = {
    "BATCH_SIZE"    : BATCH_SIZE,
    "LEARNING_RATE" : LEARNING_RATE,
    "TOTAL_EPOCH"   : TOTAL_EPOCH,
    "SAVE_PATH"     : SAVE_PATH
}

df = pd.DataFrame(data_dict, index = [0])

In [ ]:
df.to_csv("./saved_models/meta_data.csv")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
enc.eval()
latent,labels = [],[]
for i in range(len(only_normal_dataset)):
    x, label = only_normal_dataset[i]
    z = enc(x.to(DEVICE).unsqueeze(0))
    latent.append(z.cpu().detach().numpy()[0])
    labels.append(label)

pwriter = SummaryWriter(
    'runs/total_latent'
)


In [ ]:
import numpy as np
pwriter.add_embedding(
    np.array(latent[:500]),
    metadata = labels[:500],#["female", "male"],
    #label_img=sample_images[:,:,:256,:],
    global_step = g_step
)